In [1]:
import pandas as pd
import numpy as np

In [2]:
%cd /Users/alex/PETsARD

/Users/alex/PETsARD


In [3]:
from PETsARD import (
    Loader,
    Processor,
    Synthesizer
)


load = Loader(
    filepath='benchmark://adult',
    na_values={k: '?' for k in [
        'workclass',
        'occupation',
        'native-country'
    ]}
)
load.load()

/Users/alex/miniforge3/envs/dev310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loader - Benchmarker: file benchmark/adult.csv already exist and match SHA-256.
                      PETsARD will ignore download and use local data directly.


In [9]:
load.data = load.data.loc[:1000, :]

In [4]:
import pprint


def issue332(
        load: Loader,
        synthesizing_method: str,
        scaler_inhibit: bool = False
):
    pp = pprint.PrettyPrinter(depth=2)

    proc_discretizing = Processor(
        metadata=load.metadata,
    )

    if scaler_inhibit:
        print("Preproc config of Scaler before update as ...")
        pp.pprint(proc_discretizing.get_config()['scaler'])
        proc_discretizing.update_config(
            {'scaler': {
                col: None for col in load.data.columns
            }}
        )
        print("Preproc config of Scaler before after as ...")
        pp.pprint(proc_discretizing.get_config()['scaler'])

    proc_discretizing.fit(
        data=load.data,
        sequence=[
            'missing',
            'outlier',
            'scaler',
            'discretizing'
        ]
    )
    preproc_discretizing_data = proc_discretizing.transform(
        data=load.data
    )
    print("Preproc data as ...")
    print(preproc_discretizing_data.head(1))


    syn = Synthesizer(
        method=synthesizing_method,
        epsilon=10.0,
    )
    syn.create(data=preproc_discretizing_data)
    syn.fit_sample()
    print("Sync data as ...")
    print(syn.data_syn.head(1))

In [11]:
issue332(
    load=load,
    synthesizing_method='smartnoise-mst',
)

Preproc data as ...
   age workclass fnlwgt education educational-num marital-status occupation  \
0  0.0         2    0.0         1             0.0              4          6   

  relationship race gender capital-gain capital-loss hours-per-week  \
0            3    2      1          0.0          0.0            0.0   

  native-country income  
0             25      0  
age                category
workclass          category
fnlwgt             category
education          category
educational-num    category
marital-status     category
occupation         category
relationship       category
race               category
gender             category
capital-gain       category
capital-loss       category
hours-per-week     category
native-country     category
income             category
dtype: object
Synthesizer (SmartNoise): Fitting mst.
Synthesizer (SmartNoise): Fitting  mst spent 41.2776 sec.
Synthesizer (SmartNoise): Sampling mst # 558 rows (same as raw) in 0.0326 sec.
Sync data as ...

In [12]:
a = np.array([1,2,3])

In [15]:
pd.Categorical(a).to_numpy()

array([1, 2, 3])